Volodymyr Lut, UCU 2019

### Data overview

In this project the data from the [Ukrainian Open Data Portal](https://data.gov.ua) was used.

The dataset contains information about international bus routes and bus stops in Ukraine. Under [this link](https://data.gov.ua/dataset/d5d79cf2-ccc7-40a9-9b7f-4c5cd0d061f9) two files are allocated: `international_routes_stops3.csv` (contains information about stops and schedule of international routes) and `international_routes.csv` (contains information about starting and destination points of the routes, along with schedules). Copies of those datasets (actual on Monday, June 10 2019) are contained under the `raw_data` directory of this repository.

In [68]:
import pandas as pd

In [69]:
stops = pd.read_csv('./raw_data/international_routes_stops3.csv', sep=';', encoding='windows-1251')
stops.head()

,id,rejs_number,stop_city_name,stop_address,stop_code,route_frequency,distance_direct,arrival_direct,departure_direct,time_diff_direct,arrival_return,departure_return,time_diff_return,stop_number,Unnamed: 14
0,CZ1357,1,Київ,"вулиця Набережно-Печерська дорога, 10-а",3200000000,Щоденно,0,20:00,20:10,0,15:30,NaN,0,0.0,NaN
1,CZ1357,1,Київ,"вул. С.Петлюри, 32",3200000000,Щоденно,12,20:30,21:00,0,14:40,15:00,0,1.0,NaN
2,CZ1357,1,Житомир,"вул. Київська, 93",1810100000,Щоденно,140,23:20,23:40,0,12:50,13:00,0,2.0,NaN
3,CZ1357,1,Рівне,"вул. Київська, 40",5610100000,Щоденно,328,2:50,3:10,0,10:30,10:40,0,3.0,NaN
4,CZ1357,1,Львів,"вул. Стрийська, 109",4610100000,Щоденно,539,6:10,6:30,0,8:35,8:50,0,4.0,NaN


In [70]:
routes = pd.read_csv('./raw_data/international_routes.csv')
routes.head()

,permision_delivery_date,n/o,id,march_route_name,company_name,company_name_external,border_crossing,border_crossing_alternative,march_route_time_direct,march_route_time_return,march_route_regularity,license_terms,destionation_country_name,destionation_country_code,march_route_status
0,NaN,3,MD3,"АК АЕРОПОРТ БОРИСПІЛЬ (вул. Бориспільська, 7) ...",ФОП Гергі Є.М.,"S.R.L. ""SARVALTEH AUTO""",Табаки,16:00,16:00,NaN,Щоденно,2020-02-08 00:00:00,Молдова,MD,True
1,NaN,8,MD8,"Балта АС (вул. Уварова, 38а) - Дубосари","ПП Фаворіт 65005, м. Одеса, вул. Бугаївська, 2...","ОАО ""Дубоссарское автотранспортное предприятие...",Платоново,NaN,NaN,NaN,NaN,2020-06-07 00:00:00,Молдова,MD,True
2,05.06.2018,11,RU11,Бахмут-Москва,"ТОВ""Форум Авто""|ФОП""Мелкумян Ю.О.""","ООО""Газавторемонт""",Гоптівка,Плетенівка,16:50,16:50,Щоденно,2019-07-20 00:00:00,Росія,RU,True
3,NaN,12,RU12,"Бахмут (АС вул. Космонавтів,5)-Москва (Новоясе...","ТОВ ""СЕВЕР-АВТО""",без паритету,Плетенівка,NaN,NaN,NaN,NaN,2019-02-28 00:00:00,Росія,RU,True
4,NaN,13,RU13,Бахму - Москва,ФОП Базілєвський В.О.,"ООО ""Горизон-Тур""",Танюшівка,NaN,19:30,15:30,Щоденно,2019-01-07 00:00:00,Росія,RU,True


In this project file `international_routes_stops3.csv` will be used. It should be filtered - in this research I'm interested only in routes, which are heading to Poland. 

**Explanation of useful columns:**

1. *ID* decodes the bus line id - when checking this ID in the `international_routes.csv` dataset, one could determine line operator's comoany name, information about it's license, border crossing point etc.

2. *stop_city_name* together with *stop_address* will be used to determine geolocation of the stop, which will be used for visualisation on maps.

3. *stop_number* is a column which will be used to determine the position of the stop in the route.

In [72]:
### Filtering out Polish routes
pol = stops[stops.id.str[:2].str.upper() == 'PL']
pol = pol.dropna(subset=['stop_city_name', 'stop_address'])
pol.shape

(6446, 15)

In [73]:
import numpy as np

In [74]:
np.unique(pol.id).shape

(482,)

There are **6446 stops** belonging to **482 bus lines** heading to Poland after clearing the data.

### Data Mining

For visualizations we need to know the exact location of every bus stop. In order to scrape this data, [Google Maps API](https://developers.google.com/maps) will be used.

In [75]:
pol = pol.drop("Unnamed: 14", axis=1)
pol = pol.assign(full_address=(pol.stop_city_name + ', ' + pol.stop_address).values)
pol.head()

,id,rejs_number,stop_city_name,stop_address,stop_code,route_frequency,distance_direct,arrival_direct,departure_direct,time_diff_direct,arrival_return,departure_return,time_diff_return,stop_number,full_address
2530,PL944,1,Жовква,пл. Коновальця 1,4622710100,Щоденно,0,4:20,4:40,0,14:55,NaN,0,0.0,"Жовква, пл. Коновальця 1"
2531,PL944,1,Рава Руська,"вул. Двірцева, 4",4622710400,Щоденно,32,5:25,5:30,0,14:05,14:10,0,1.0,"Рава Руська, вул. Двірцева, 4"
2534,PL944,1,Томашів Любельський,"ul. Zamojska, 9",22-600,Щоденно,63,6:05,NaN,-1,11:00,11:30,0,4.0,"Томашів Любельський, ul. Zamojska, 9"
2535,PL944,2,Жовква,пл. Коновальця 1,4622710100,Щоденно,0,21:50,22:10,0,6:25,NaN,0,5.0,"Жовква, пл. Коновальця 1"
2536,PL944,2,Рава Руська,"вул. Двірцева, 4",4622710400,Щоденно,32,22:55,23:00,0,5:35,5:40,0,6.0,"Рава Руська, вул. Двірцева, 4"


In [76]:
np.unique(pol.full_address).shape

(2080,)

In [77]:
import googlemaps
import re
# Uncomment line below and paste your key there
# gmaps = googlemaps.Client(key='<YOUR KEY HERE>')

In [78]:
from tqdm import tqdm

In [79]:
# Use this code to scrape geocoding data and save it to file
geocoded_addresses = []
for address in tqdm(np.unique(pol.full_address)):
    geocode_result = gmaps.geocode(address)
    geocoded_addresses.append(geocode_result)
np.savetxt("geocoded.tmp", geocoded_addresses, fmt='%s')

  0%|          | 0/2080 [00:00<?, ?it/s]


NameError: name 'gmaps' is not defined

In [80]:
import json

In [81]:
# Use this code to restore geocoding data from file
with open('geocoded.tmp') as fp:  
    geocoded_addresses = []   
    geocode_result = fp.readline()
    i = 0
    while geocode_result:
        geocode_result = geocode_result.replace("Boryspil'", "Boryspil")
        geocode_result = re.sub("(?<=[a-z])'(?=[a-z])", "", geocode_result, flags=re.IGNORECASE).replace("True", '"true"').replace("False", '"false"').replace("'", '"').replace('""', '"').replace('"-', '-')
        geocode_result = re.sub('(?<=[a-z]) "(?=[a-z])', " ", geocode_result, flags=re.IGNORECASE)
        geocode_result = re.sub('(?<=[a-z])" (?=[a-z])', " ", geocode_result, flags=re.IGNORECASE)
        geocode_result = re.sub('(?<=[a-z])", (?=[a-z])', " ", geocode_result, flags=re.IGNORECASE)
        if(i == 735):
            print(geocode_result)
        geocoded_addresses.append(json.loads(geocode_result))
        geocode_result = fp.readline()

In [112]:
full_addresses = pol.full_address
unique_addresses = np.unique(full_addresses).tolist()
geolocation_list = []
for address in tqdm(full_addresses):
    i = unique_addresses.index(address)
    try :
        lat = geocoded_addresses[i][0]['geometry']['location']['lat']
        lng = geocoded_addresses[i][0]['geometry']['location']['lng']
        country = geocoded_addresses[i][0]['address_components'][-2]['long_name']
        administrative_level_1 = geocoded_addresses[i][0]['address_components'][-3]['long_name']
        administrative_level_2 = geocoded_addresses[i][0]['address_components'][-4]['long_name']
        geolocation_list.append([
            lat,
            lng,
            country,
            administrative_level_1,
            administrative_level_2
        ])
    except IndexError :
        geolocation_list.append([None, None, None, None])
        print (address, i)

geocoded_dataframe = pd.DataFrame(geolocation_list, columns = ['lat', 'lng', 'country', 'administrative_level_1', 'administrative_level_2'])

100%|██████████| 6446/6446 [00:00<00:00, 30577.27it/s]

Ельбонг, ul. Grunwaldzka, 14 490
Минськ Мазовецький, pl.Dworcowa,1 1208
Замостя, вул. Грубешівська, 1 557
Червінськ-над-Віслою, вул. В. Ягели, 18 1819
Монастирська, вул. Шевченка, 57 1212
Пила, ul. Kvitova, 18 1376
м. Грудзьонз, ul. Dworcowa, 1 1950
Нови-Двур-Мазовецьки "Модлин", ul.gen.Wiktora Thommee 1241
Нови-Двур-Мазовецьки "Модлин", ul.gen.Wiktora Thommee 1241
Петрикув Трибунальський, вул.Польської Організації Військової 1370
Монастирська, вул. Шевченка, 57 1212
м. Грудзьонз, ul. Dworcowa, 1 1950
Нови-Двур-Мазовецьки "Модлин", ul.gen.Wiktora Thommee 1241
Бердичів, пл. Привокзальна, 1-А 49
вул.Звірки і Вігури,1, Dw. PKS "Zachodnia", str. Al. Jerozlimskie,144 1925
Грудзонз, ul. Rapatskogo 33 428
Нови-Двур-Мазовецьки "Модлин", ul.gen.Wiktora Thommee 1241
Нови-Двур-Мазовецьки "Модлин", ul.gen.Wiktora Thommee 1241
Грудзонз, ul. Rapatskogo 33 428
Ельбонг, plac Dworcowy 489
Нови-Двур-Мазовецьки "Модлин", ul.gen.Wiktora Thommee 1241
Грудзонз, Dworcowa,42 427
Грудзенз, Dworcowa,42 425
Крот

(6446, 5)

In [121]:
poland_geocoded = pd.concat([pol.reset_index(drop=True), geocoded_dataframe.reset_index(drop=True)], axis=1)

In [124]:
poland_geocoded.head()

,id,rejs_number,stop_city_name,stop_address,stop_code,route_frequency,distance_direct,arrival_direct,departure_direct,time_diff_direct,arrival_return,departure_return,time_diff_return,stop_number,full_address,lat,lng,country,region,city
0,PL944,1,Жовква,пл. Коновальця 1,4622710100,Щоденно,0,4:20,4:40,0,14:55,NaN,0,0.0,"Жовква, пл. Коновальця 1",50.056643,23.972478,Ukraine,Lvivska oblast,Zhovkivskyi district
1,PL944,1,Рава Руська,"вул. Двірцева, 4",4622710400,Щоденно,32,5:25,5:30,0,14:05,14:10,0,1.0,"Рава Руська, вул. Двірцева, 4",50.230070,23.636844,Ukraine,Lvivska oblast,Zhovkivskyi district
2,PL944,1,Томашів Любельський,"ul. Zamojska, 9",22-600,Щоденно,63,6:05,NaN,-1,11:00,11:30,0,4.0,"Томашів Любельський, ul. Zamojska, 9",50.454665,23.419679,Poland,lubelskie,tomaszowski
3,PL944,2,Жовква,пл. Коновальця 1,4622710100,Щоденно,0,21:50,22:10,0,6:25,NaN,0,5.0,"Жовква, пл. Коновальця 1",50.056643,23.972478,Ukraine,Lvivska oblast,Zhovkivskyi district
4,PL944,2,Рава Руська,"вул. Двірцева, 4",4622710400,Щоденно,32,22:55,23:00,0,5:35,5:40,0,6.0,"Рава Руська, вул. Двірцева, 4",50.230070,23.636844,Ukraine,Lvivska oblast,Zhovkivskyi district


In [118]:
poland_geocoded.to_csv('poland_bus_routes_geocoded.csv')

,lat,lng,country,region,city
0,50.056643,23.972478,Ukraine,Lvivska oblast,Zhovkivskyi district
1,50.230070,23.636844,Ukraine,Lvivska oblast,Zhovkivskyi district
2,50.454665,23.419679,Poland,lubelskie,tomaszowski
3,50.056643,23.972478,Ukraine,Lvivska oblast,Zhovkivskyi district
4,50.230070,23.636844,Ukraine,Lvivska oblast,Zhovkivskyi district
5,50.454665,23.419679,Poland,lubelskie,tomaszowski
6,50.056643,23.972478,Ukraine,Lvivska oblast,Zhovkivskyi district
7,50.230070,23.636844,Ukraine,Lvivska oblast,Zhovkivskyi district
8,50.454665,23.419679,Poland,lubelskie,tomaszowski
9,50.056643,23.972478,Ukraine,Lvivska oblast,Zhovkivskyi district


In [117]:
full_addresses

2530                              Жовква, пл. Коновальця 1
2531                         Рава Руська, вул. Двірцева, 4
2534                  Томашів Любельський, ul. Zamojska, 9
2535                              Жовква, пл. Коновальця 1
2536                         Рава Руська, вул. Двірцева, 4
2539                  Томашів Любельський, ul. Zamojska, 9
2540                              Жовква, пл. Коновальця 1
2541                         Рава Руська, вул. Двірцева, 4
2544                  Томашів Любельський, ul. Zamojska, 9
2545                              Жовква, пл. Коновальця 1
2546                         Рава Руська, вул. Двірцева, 4
2549                  Томашів Любельський, ul. Zamojska, 9
2550                             Київ, вул. С. Петлюри, 32
2551                            Житомир, вул. Київська, 93
2552                            Вінниця, вул. Киїівська, 8
2553                        Вінниця, Хмельницьке шосе, 107
2554                         Летичів, вул. Ю.Савицького,